# 시스템 트레이딩 리서치 리포트 (Jupyter Notebook)

**작성일:** 2025-08-18 23:45:31

이 노트북은 *시가/지표 변화율 기반 전략*을 재현하고, 백테스트 및 리포팅까지 한 번에 수행할 수 있는 **퀀트 리서치 템플릿**입니다.

## 리포트 구성
1. 환경 설정 & 파라미터  
2. 데이터 로드/전처리  
3. 전략 로직 (시그널/손절)  
4. 백테스트 (거래 로그/일별 손익)  
5. 성과 지표 (Sharpe, MDD, CAGR 등)  
6. 시각화 (에쿼티 커브, 드로다운)  
7. 결과물 Export (엑셀 리포트)

> **사용 전 준비물:** CSV 데이터 파일(`mt-10yr-new.csv`)이 노트북과 같은 폴더에 있어야 합니다.


## 1) 환경 설정

In [ ]:
# 표준 라이브러리 & 데이터 과학 스택
import os
import math
from datetime import datetime

import numpy as np
import pandas as pd

# 시각화: 지침상 seaborn 사용 금지, matplotlib만 사용
import matplotlib.pyplot as plt

# 노트북 표시 옵션
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 120)

print('Env ready:', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

## 2) 파라미터

In [ ]:
# 데이터 파일명
file_name = "mt-10yr-new.csv"

# 전략 파라미터 (필요시 수정)
rising_rate  = 2.460   # 상승 구간 조정률
falling_rate = 2.520   # 하락 구간 조정률

# 밴드 (열린구간; idx는 Index6_Pct_Change)
a, b = 0.20, 1.31      # 상승: a < idx < b
d, c = -1.05, -0.00    # 하락: d < idx < c

# 스탑로스 (0.004=0.4%), 틱 0.1
sl_buy  = 0.004
sl_sell = 0.0165
ratio_is_percent = False
tick = 0.1

print('Parameters loaded.')

## 3) 데이터 로드/전처리

In [ ]:
# CSV 로드
df = pd.read_csv(file_name, header=None)
df.columns = [
    "Date", "Open", "High", "Low", "Close",
    "Index1", "Index2", "Index3", "Index4", "Index5",
    "Index6", "Index7", "Index8", "Index9", "Index10", "Index11"
]

# 날짜 정규화 (시간 제거)
df["Date"] = pd.to_datetime(df["Date"]).dt.normalize()

def pct_change_today_over_yday(s, cur, prev):
    # (금일값 / 전일값_전일 - 1) * 100
    return (s[cur] / s[prev].shift(1) - 1) * 100

# 시가/Index6 변동률 + 전일 종가
df["Open_Pct_Change"]   = pct_change_today_over_yday(df, "Open",  "Close")
df["Index6_Pct_Change"] = pct_change_today_over_yday(df, "Index6","Index6")
df["Prev_Close"]        = df["Close"].shift(1)

need_cols = ["Date","Open","High","Low","Close","Prev_Close","Open_Pct_Change","Index6_Pct_Change"]
df_bt = df.loc[:, need_cols].dropna().reset_index(drop=True).copy()

print('Data shape:', df.shape, 'Backtest frame:', df_bt.shape)

## 4) 전략 유틸/로직

In [ ]:
def ceil_to_nearest(x, step):
    return np.ceil(x / step) * step

def stop_amount(prev_close, ratio, ratio_is_percent=False, tick=0.1):
    amt = prev_close * (ratio*0.01 if ratio_is_percent else ratio)
    return ceil_to_nearest(amt, tick)

def generate_trades_log(
    data,
    rising_rate, falling_rate,
    a, b, d, c,
    sl_buy, sl_sell,
    ratio_is_percent=False, tick=0.1
):
    logs = []
    cols = ["Date","Open","High","Low","Close","Prev_Close","Open_Pct_Change","Index6_Pct_Change"]
    for Date, O, H, L, C, PrevC, opct, idx in data[cols].itertuples(index=False, name=None):
        side = None
        band = None
        adj  = None
        stop = np.nan
        stop_hit = False
        pnl = 0.0

        # 상승 밴드 (열린구간)
        if (idx > 0) and (a < idx < b):
            band = "RISING"
            adj = idx * rising_rate
            if opct > adj:  # BUY
                side = "BUY"
                stop = O - stop_amount(PrevC, sl_buy,  ratio_is_percent, tick)
                pnl = (stop - O) if round(L,3) <= round(stop,3) else (C - O)
                stop_hit = round(L,3) <= round(stop,3)
            elif opct < adj:  # SELL
                side = "SELL"
                stop = O + stop_amount(PrevC, sl_sell, ratio_is_percent, tick)
                pnl = (O - stop) if round(H,3) >= round(stop,3) else (O - C)
                stop_hit = round(H,3) >= round(stop,3)

        # 하락 밴드 (열린구간)
        elif (idx < 0) and (d < idx < c):
            band = "FALLING"
            adj = idx * falling_rate
            if opct < adj:  # SELL
                side = "SELL"
                stop = O + stop_amount(PrevC, sl_sell, ratio_is_percent, tick)
                pnl = (O - stop) if round(H,3) >= round(stop,3) else (O - C)
                stop_hit = round(H,3) >= round(stop,3)
            elif opct > adj:  # BUY
                side = "BUY"
                stop = O - stop_amount(PrevC, sl_buy,  ratio_is_percent, tick)
                pnl = (stop - O) if round(L,3) <= round(stop,3) else (C - O)
                stop_hit = round(L,3) <= round(stop,3)

        if side is None:
            continue

        logs.append({
            "Date": Date,
            "Band": band,
            "IndexChange(%)": idx,
            "OpenPct(%)": opct,
            "AdjThreshold(%)": adj,
            "Side": side,
            "Entry": O,
            "Close": C,
            "Low": L,
            "High": H,
            "PrevClose": PrevC,
            "Stop": stop,
            "StopHit": stop_hit,
            "PnL": float(pnl),
        })

    trades = pd.DataFrame(logs).sort_values("Date").reset_index(drop=True)
    return trades

def make_daily_pnl_all_dates(df_all_dates, trades_df):
    all_dates = (
        df_all_dates[["Date"]]
        .drop_duplicates()
        .sort_values("Date")
        .reset_index(drop=True)
    )

    if trades_df.empty:
        daily = all_dates.copy()
        daily["DailyPnL"] = 0.0
        daily["Trades"]   = 0
        daily["CumPnL"]   = 0.0
        return daily

    byday = (
        trades_df.groupby("Date", as_index=False)
        .agg(DailyPnL=("PnL", "sum"), Trades=("PnL", "size"))
    )

    daily = all_dates.merge(byday, on="Date", how="left")
    daily["DailyPnL"] = daily["DailyPnL"].fillna(0.0)
    daily["Trades"]   = daily["Trades"].fillna(0).astype(int)
    daily["CumPnL"]   = daily["DailyPnL"].cumsum()
    return daily

def _fmt(x, nd=2):
    s = f"{x:.{nd}f}"
    return s.replace("-0.00", "0.00").replace("-0.0", "0.0")

print('Strategy functions loaded.')

## 5) 백테스트 실행

In [ ]:
trades_df = generate_trades_log(
    df_bt,
    rising_rate, falling_rate,
    a, b, d, c,
    sl_buy, sl_sell,
    ratio_is_percent=ratio_is_percent, tick=tick
)

daily_df = make_daily_pnl_all_dates(df, trades_df)

print('Trades:', len(trades_df), 'Days:', len(daily_df))
trades_df.head(10)

## 6) 성과 지표 계산

In [ ]:
def performance_stats(daily_df, trades_df):
    stats = {}
    stats['total_trades'] = int(trades_df.shape[0])
    stats['total_pnl'] = float(trades_df['PnL'].sum()) if not trades_df.empty else 0.0
    stats['avg_pnl_per_trade'] = float(trades_df['PnL'].mean()) if not trades_df.empty else 0.0
    stats['win_rate'] = (trades_df['PnL'] > 0).mean() if not trades_df.empty else 0.0

    # 일별 손익 기반 샤프, CAGR, MDD
    dd = daily_df.copy()
    dd = dd.sort_values('Date').reset_index(drop=True)

    # 일수 및 연환산 가정 (거래일 252일)
    if len(dd) > 1:
        days = (dd['Date'].iloc[-1] - dd['Date'].iloc[0]).days
    else:
        days = 0
    years = days / 365.25 if days > 0 else 0

    # 일별 수익률 가정: PnL 단위가 절대금액이면 비율 필요.
    # 여기서는 PnL 자체의 변동성으로 샤프를 단순 근사(스케일 조정 필요 시 사용자 정의)
    ret = dd['DailyPnL'].values
    if ret.size > 1 and ret.std(ddof=1) != 0:
        sharpe_daily = ret.mean() / ret.std(ddof=1)
        sharpe_annual = sharpe_daily * np.sqrt(252)
    else:
        sharpe_daily = 0.0
        sharpe_annual = 0.0

    # 에쿼티 커브 기반 MDD
    equity = dd['CumPnL'].values
    if equity.size > 0:
        peak = np.maximum.accumulate(equity)
        drawdown = equity - peak
        mdd = drawdown.min() if drawdown.size > 0 else 0.0
    else:
        mdd = 0.0

    # CAGR: 절대 PnL로는 정의 어려움. 초기자본 1로 가정 시 구현 예시
    initial_capital = 1.0
    final_capital = initial_capital + (dd['CumPnL'].iloc[-1] if len(dd) else 0.0)
    cagr = (final_capital / initial_capital) ** (1/years) - 1 if years > 0 and final_capital > 0 else 0.0

    stats.update({
        'sharpe_annual(approx)': float(sharpe_annual),
        'mdd_abs': float(mdd),
        'cagr_assuming_init1': float(cagr),
        'period_days': int(days),
        'period_years': float(years),
    })
    return pd.DataFrame([stats])

perf = performance_stats(daily_df, trades_df)
perf

## 7) 시각화

In [ ]:
# 에쿼티 커브
plt.figure()
plt.plot(daily_df['Date'], daily_df['CumPnL'])
plt.title('Equity Curve (CumPnL)')
plt.xlabel('Date')
plt.ylabel('CumPnL')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 드로다운
equity = daily_df['CumPnL'].values
if equity.size > 0:
    peak = np.maximum.accumulate(equity)
    drawdown = equity - peak
else:
    drawdown = np.array([])

plt.figure()
if drawdown.size > 0:
    plt.plot(daily_df['Date'], drawdown)
plt.title('Drawdown (Absolute)')
plt.xlabel('Date')
plt.ylabel('Drawdown')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 8) 결과물 Export (엑셀 리포트)

In [ ]:
out_path = "mt_kosdaq_trades.xlsx"
with pd.ExcelWriter(out_path, engine="openpyxl") as writer:
    trades_df.to_excel(writer, sheet_name="trades", index=False)
    daily_df.to_excel(writer, sheet_name="daily_pnl", index=False)
    perf.to_excel(writer, sheet_name="performance", index=False)

print(f"엑셀 저장 완료: {out_path}")

---

### 📌 운영 팁
- **거래비용/슬리피지** 열을 추가하여 현실 반영 (수수료/세금/스프레드)  
- **워크포워드 검증**: 기간 분할(훈련/검증) + 파라미터 고정 후 검증  
- **파라미터 스윕**: grid/random/Bayes로 `rising_rate`, `falling_rate`, `a,b,c,d`, `sl_*` 민감도 분석  
- **유닛 테스트**: 임계치/손절 판정 로직에 대한 단위 테스트로 재현성 강화  
- **버전 스냅샷**: 데이터 버전, 파라미터, 코드 해시를 함께 저장

> 성과/리스크 산출 방식은 **절대 PnL vs 비율 수익률**에 따라 다릅니다. 실제 운용에서는 초기자본/포지션 사이즈/거래비용을 명시하고 퍼포먼스 지표를 계산하세요.
